EDA

In [1]:
import pandas as pd
import numpy as np

In [18]:
df = pd.read_csv('../data/ca_real_estate.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Price       5000 non-null   int64 
 1   Bedrooms    5000 non-null   int64 
 2   Bathrooms   5000 non-null   int64 
 3   SqFt        5000 non-null   int64 
 4   City        5000 non-null   object
 5   Province    5000 non-null   object
 6   Year_Built  5000 non-null   int64 
 7   Type        5000 non-null   object
 8   Garage      5000 non-null   int64 
 9   Lot_Area    5000 non-null   int64 
dtypes: int64(7), object(3)
memory usage: 390.8+ KB


In [4]:
df.head()

,Price,Bedrooms,Bathrooms,SqFt,City,Province,Year_Built,Type,Garage,Lot_Area
0,873630,5,2,1010,Montreal,BC,1960,Condo,1,7919
1,377869,2,2,3591,Toronto,ON,1958,House,1,7304
2,128030,4,1,3823,Montreal,ON,2002,House,0,4548
3,117730,3,2,2848,Montreal,QC,1975,Apartment,1,3374
4,292476,4,1,3659,Vancouver,QC,2018,Condo,1,1281


In [8]:
df.isnull().sum()

Price         0
Bedrooms      0
Bathrooms     0
SqFt          0
City          0
Province      0
Year_Built    0
Type          0
Garage        0
Lot_Area      0
dtype: int64

In [6]:
df.shape

(5000, 10)

In [19]:
df = df.drop(columns=['Province'])

In [20]:
df.shape

(5000, 9)

ValueError: could not convert string to float: 'Montreal'

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Price       5000 non-null   int64 
 1   Bedrooms    5000 non-null   int64 
 2   Bathrooms   5000 non-null   int64 
 3   SqFt        5000 non-null   int64 
 4   City        5000 non-null   object
 5   Year_Built  5000 non-null   int64 
 6   Type        5000 non-null   object
 7   Garage      5000 non-null   int64 
 8   Lot_Area    5000 non-null   int64 
dtypes: int64(7), object(2)
memory usage: 351.7+ KB


FEATURE ENCODING

In [21]:
df = pd.get_dummies(df, columns=['City', 'Type'], drop_first=True).astype(int)

In [22]:
df.head()

,Price,Bedrooms,Bathrooms,SqFt,Year_Built,Garage,Lot_Area,City_Montreal,City_Ottawa,City_Toronto,City_Vancouver,Type_Condo,Type_House
0,873630,5,2,1010,1960,1,7919,1,0,0,0,1,0
1,377869,2,2,3591,1958,1,7304,0,0,1,0,0,1
2,128030,4,1,3823,2002,0,4548,1,0,0,0,0,1
3,117730,3,2,2848,1975,1,3374,1,0,0,0,0,0
4,292476,4,1,3659,2018,1,1281,0,0,0,1,1,0


In [23]:
print(df.dtypes)


Price             int32
Bedrooms          int32
Bathrooms         int32
SqFt              int32
Year_Built        int32
Garage            int32
Lot_Area          int32
City_Montreal     int32
City_Ottawa       int32
City_Toronto      int32
City_Vancouver    int32
Type_Condo        int32
Type_House        int32
dtype: object


In [24]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_absolute_error

# Define features and target
X = df.drop(columns=['Price'])  # Features
y = df['Price']  # Target variable

# Split data (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [28]:
y_test.shape

(1000,)

In [29]:
param_grid = {
    'n_estimators': [100, 300, 500], 
    'learning_rate': [0.01, 0.1, 0.2], 
    'max_depth': [3, 5, 7],  
    'min_child_weight': [1, 3, 5], 
    'subsample': [0.8, 1.0],  
    'colsample_bytree': [0.8, 1.0]  
}

xgb_reg = xgb.XGBRegressor(objective='reg:squarederror', random_state=42)

grid_search = GridSearchCV(
    estimator=xgb_reg,
    param_grid=param_grid,
    scoring='neg_mean_absolute_error',
    cv=3,  # 3-fold cross-validation
    verbose=1,
    n_jobs=-1  # Use all CPU cores
)

grid_search.fit(X_train, y_train)

print("Best Hyperparameters:", grid_search.best_params_)


Fitting 3 folds for each of 324 candidates, totalling 972 fits
Best Hyperparameters: {'colsample_bytree': 0.8, 'learning_rate': 0.01, 'max_depth': 3, 'min_child_weight': 5, 'n_estimators': 100, 'subsample': 0.8}
